<a href="https://colab.research.google.com/github/amolaka/DS-3001---Voting-Project/blob/main/voting_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
! pip3 install mapclassify

In [12]:
# Import libraries and packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import mapclassify
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [30]:
# Import dataset
# ! git clone https://github.com/DS3001/project_voting
#data = pd.read_csv('/content/project_chd/fhs_train.csv')
gdf = gpd.read_file('VirginiaCounty_ClippedToShoreline.shx') # Load Virginia shapefile for maps
vdf = pd.read_csv('/content/project_voting/data/voting_VA.csv') # Load historical voting data
cdf = pd.read_csv('/content/project_voting/data/county_adjacencies.csv') # Load county adjacency data and population data

# Merge shapefile data and county adjacency data:
gdf['FIPS_left'] = pd.to_numeric(gdf['STCOFIPS'])
df = gdf.merge(cdf,left_on='FIPS_left',right_on='FIPS')  # merge county population and adjacency data into df
del gdf, cdf

KeyError: 'STCOFIPS'

In [ ]:
# Choropleth map